+ PKG 規格自動關閉
+ 目前庫存出貨比率 出貨/[目前庫存]
+ 入庫 / [出貨+入庫]
+ 3 個月內出貨 bin格數量 , 1 個月內入庫 bin 格
+ bin 格數量比對 , 出貨與入庫數量百分比
+ 2019/02/26 新增 bincode 清單
+ 拆分 Bincode 
+ 合併 N/Z 料號
+ 特殊料號處理
+ Insert Planing
+ 庫齡 < 14天不列入review, & 90 未入庫 
+ 檢查已經關閉
+ 邏輯修正 rawData Combine Group & List Max Age
+ 調整Risk Bin

In [1]:
#Load DB 元件
FilePath="D://AP//PythonLen//PKG_SPEC_TurnRate//"
import os
import pandas as pd
import numpy as np 
from dateutil.relativedelta import relativedelta
from datetime import datetime
os.chdir(FilePath)

In [2]:
# Lxtar component
import ConnectDB as cn
import SqlPkgBinQty as sqlstr
import MailTool as MyMail

In [3]:
# 取得 SQL 
sSql=sqlstr.SqlStr()
#取得區間 
EndDate=datetime.now()
#下線 90D , 
StarDate=EndDate+relativedelta(months=-3)
#入庫 30D
StarDateM=EndDate+relativedelta(months=-1)
EndDate=EndDate.strftime('%Y/%m/%d')
StarDate=StarDate.strftime('%Y/%m/%d')
StarDateM=StarDateM.strftime('%Y/%m/%d')

In [4]:
print('StarDate',StarDate)
print('StarDateM',StarDateM)
print('EndDate',EndDate)

StarDate 2019/06/02
StarDateM 2019/08/02
EndDate 2019/09/02


ItemCost 取消
strSql=sSql.ItemCost(EndDate)
dbCon=cn.OracCn()
dfItemCost=dbCon.ExecMSSql_Commend(dbCon.lexmsdb6,'RPT',strSql)

In [5]:
#產品資訊
strSql=sSql.ModelProp()
dbCon=cn.OracCn()
dtModelPro=dbCon.Exec_OracleCommend(dbCon.T01HIS1,strSql)

In [6]:
# 入庫資料 now-30D
#strSql=sSql.GepInVInput(StarDate,EndDate)
strSql=sSql.GepInVInput(StarDateM,EndDate)
dbCon=cn.OracCn()
dtInvin=dbCon.Exec_OracleCommend(dbCon.T05HIS01,strSql)

In [7]:
# 下線資料 , 出貨 5 碼 bin 是否過濾 , now-90D
strSql=sSql.GetShip(StarDate,EndDate)
dtShip=dbCon.Exec_OracleCommend(dbCon.T05HIS01,strSql)
dtShip=dtShip.groupby(['PRODUCTNO','BINCODE']).sum()
dtShip=dtShip.reset_index()

In [8]:
#工單領用 cz now-90D
strSql=sSql.GetIssueWO(StarDate,EndDate)
dtISWo=dbCon.Exec_OracleCommend(dbCon.C01HIS01,strSql)

In [9]:
#目前 WHMS 庫存 -now-90D
strSql=sSql.GetNOH()
dbCon=cn.OracCn()
dtNOH_T05=dbCon.Exec_OracleCommend(dbCon.T05HIS01,strSql)
dtNOH_C01=dbCon.Exec_OracleCommend(dbCon.C01HIS01,strSql)
dtNOH_S01=dbCon.Exec_OracleCommend(dbCon.S01HIS01,strSql)

In [10]:
#Merage C01 & TW INV
dtNOH_C01=pd.merge(dtNOH_C01,dtNOH_S01,how='outer')
dtNOH=pd.merge(dtNOH_C01,dtNOH_T05,how='outer')
del dtNOH_T05,dtNOH_C01,dtNOH_S01

In [11]:
dtNOH.columns

Index(['PRODUCTNO', 'BINCODE', 'OH_QTY'], dtype='object')

In [12]:
dtNOH=dtNOH.groupby(['PRODUCTNO', 'BINCODE']).sum()
dtNOH=dtNOH.reset_index()
#dtNOH.loc[dtNOH['BINCODE']=='30AX1EUA']

In [13]:
#test
dtNOH.loc[dtNOH['PRODUCTNO']=='95.C3030.HAD036N',['OH_QTY']].sum()

OH_QTY    180000
dtype: int64

In [14]:
# bincode 庫齡
strSql=sSql.GetNOHAGE()
dbCon=cn.OracCn()
dtNOHAGE_T05=dbCon.Exec_OracleCommend(dbCon.T05HIS01,strSql)
dtNOHAGE_C01=dbCon.Exec_OracleCommend(dbCon.C01HIS01,strSql)
dtNOHAGE_S01=dbCon.Exec_OracleCommend(dbCon.S01HIS01,strSql)
dtNOHAGE_C01=pd.merge(dtNOHAGE_C01,dtNOHAGE_S01,how='outer')
dtNOHAGE=pd.merge(dtNOHAGE_C01,dtNOHAGE_T05,how='outer')
del dtNOHAGE_T05,dtNOHAGE_C01,dtNOHAGE_S01

In [15]:
# 分組
maxAge=max(dtNOHAGE['INV_AGE'])
bins=[-1,15,30,61,91,180,360,maxAge]
#lab=["0-15","16-30",'31-60','61-90','91-180','181-360','>360']
lab=[15,30,60,90,180,360,390]
dtNOHAGE['AGE']=pd.cut(dtNOHAGE['INV_AGE'],bins,labels=lab)
#暫時取 MIN Mapping 計算
dtNOHAGE=dtNOHAGE.groupby(['PRODUCTNO','BINCODE'])['AGE'].min()
dtNOHAGE=dtNOHAGE.reset_index()
dtNOHAGE=dtNOHAGE.rename(columns={'AGE':'MIN_AGE'})

In [16]:
# Bincode 編碼方式
strSql=sSql.GetSPEC_NamingRule('95%')
dbCon=cn.OracCn()
dfNaming=dbCon.ExecMSSql_Commend(dbCon.lexmsdb2,'Planning',strSql)

In [17]:
# 工單領用
dfISSWO=dtISWo['WO_QTY'].groupby([dtISWo['PRODUCTNO'],dtISWo['BINCODE']]).sum().to_frame()
dfISSWO=dfISSWO.reset_index()

dtBin=pd.merge(dtShip,dtInvin,how='outer', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)
dtBin=pd.merge(dtBin,dfISSWO,how='outer', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)
dtBin=pd.merge(dtBin,dtNOH,how='outer', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)

In [18]:
# 主要資料 RawData
dfparbin=pd.merge(dtShip[['PRODUCTNO', 'BINCODE']],dtInvin[['PRODUCTNO', 'BINCODE']],how='outer')
dfparbin=pd.merge(dfparbin,dfISSWO[['PRODUCTNO', 'BINCODE']],how='outer')
dfparbin=pd.merge(dfparbin,dtNOH[['PRODUCTNO', 'BINCODE']],how='outer')
dfparbin['T']=0
dfparbin=dfparbin.groupby(['PRODUCTNO', 'BINCODE']).sum()
dfparbin=dfparbin.drop(columns=['T'],axis=1)
dfparbin.shape

(50481, 0)

In [19]:
dtBin=pd.merge(dfparbin,dtInvin,how='left', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)
dtBin=dtBin.merge(dtShip,how='left', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)
dtBin=pd.merge(dtBin,dfISSWO,how='left', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)
dtBin=pd.merge(dtBin,dtNOH,how='left', left_on=['PRODUCTNO','BINCODE'] , right_on=['PRODUCTNO','BINCODE']).fillna(0)

In [20]:
# Merge Product 
dtBin=pd.merge(dtBin,dtModelPro,how='left', left_on=['PRODUCTNO'] , right_on=['PART_NO'])
dtBin=dtBin.drop(['PART_NO'],axis=1)

In [21]:
#主資料 Summary , by part_no
# Group by 
dtMain=dtBin.groupby(['PRODUCTNO'],axis=0).agg('sum')
dtMain=dtMain.fillna(0)
dtMain=dtMain.reset_index()

In [22]:
# 新增 Result Columns
addCols=['INPUT_BIN','SHIP_BIN','NOH_BIN','RISK_BIN',
         'RISK1_BIN','RISK_QTY','RISK1_QTY','UNShip_Ratio','InvFree_Ratio']
for col in addCols:
    dtMain[col]=np.zeros(dtMain.shape[0])

In [23]:
#取得 BU 分類
strSql=sSql.GetBU()
dbCon=cn.OracCn()
dtBU=dbCon.Exec_OracleCommend(dbCon.T01HIS1,strSql)

In [24]:
# 增加庫齡資訊
dtBin=pd.merge(dtBin,dtNOHAGE,how='left',left_on=['PRODUCTNO','BINCODE'],
               right_on=['PRODUCTNO','BINCODE'])
dtBin=dtBin.fillna(0)

In [25]:
# 增加 Model Name Group
dtMain=pd.merge(dtMain,dtModelPro,how='left', left_on=['PRODUCTNO'] , right_on=['PART_NO'])
dtMain=dtMain.drop(['PART_NO'],axis=1)

規格處理

In [26]:
#傳入一個料號 回傳位置 
def GetLenth(dfNaming,sType):
    Pos=dfNaming[dfNaming['SPEC_TYPE']==sType]
    sPos=0
    ePos=0
    if Pos.shape[0] > 0 :
        Pos=Pos.iloc[0,:]
        sPos=int(Pos['CODE_INDEX'])
        ePos=int(Pos['CODE_LENGTH'])
    else :
        sPos=1
        ePos=0
    return {'sPos':sPos-1,'ePos':ePos}     

In [27]:
#定義要取得的屬性與編碼長度
sTypes=['WL','CIE','IV','VF']
#建立回傳Table
cols=['PRODUCTNO','WL_S','WL_E','CIE_S','CIE_E','IV_S','IV_E','VF_S','VF_E']
ParNameRule=pd.DataFrame()
#ParNameRule=pd.DataFrame(columns=cols)
sValue2={} 

pl=dfNaming.PART_NO.unique()
for p in pl:
    for t in sTypes:
        En=GetLenth(dfNaming[dfNaming['PART_NO']==p],t)
        #print(p)
    #每個料號位置
        sValue2[t]=En
    vals=[p,sValue2['WL']['sPos'],sValue2['WL']['ePos'],sValue2['CIE']['sPos'],sValue2['CIE']['ePos'],
    sValue2['IV']['sPos'],sValue2['IV']['ePos'],sValue2['VF']['sPos'],sValue2['VF']['ePos']]
    ParNameRule=ParNameRule.append(pd.DataFrame([vals]))
ParNameRule.columns=cols

合併TW & CZ 料號
95.S3806.W0A0AZN / 95.S3806.W0A0AZZ
PM 規格是否關閉
W12.3  C8N = C8Z => C8N+EHN = C8Z+EHZ

In [28]:
#合併TW & CZ 料號 , 建立 PartGroup
#95.S3806.W0A0AZN / 95.S3806.W0A0AZZ
ParGroup=dtBin['PRODUCTNO'].unique().tolist()
ParGroup=sorted(ParGroup)
dtBin['Part_Group']=dtBin['PRODUCTNO'].copy()
dtMain['Part_Group']=dtMain['PRODUCTNO'].copy()
for p in ParGroup:    
#p='95.S3806.W0A0AZN'
    if (p[len(p)-1])=='N':  
        bp=p[0:(len(p)-1)]+'Z'
        dtBin.loc[(dtBin['PRODUCTNO']==p)|(dtBin['PRODUCTNO']==bp),'Part_Group']=p
        dtMain.loc[(dtMain['PRODUCTNO']==p)|(dtMain['PRODUCTNO']==bp),'Part_Group']=p        
    #print(bp)
#dtBin.loc[dtBin['PRODUCTNO']==p]

In [29]:
# W12.3 C8N = C8Z => C8N+EHN = C8Z+EHZ 20190406
dtMain.loc[dtMain['Part_Group']=='95.S3806.W0A0EHN','Part_Group']='95.S3806.W0A0C8N'
dtBin.loc[dtBin['Part_Group']=='95.S3806.W0A0EHN','Part_Group']='95.S3806.W0A0C8N'

In [30]:
#主程式  計算BIN 數與取得 BIN CODE , 以 Part Group 計算
def GetRawCountbyGroup(dtParts,partNo='95.T4014.W0B1B2Z'):
    # 庫齡取 Max 放寬 14 天不列入    
    parms={'SHIP':'sum','INPUT':'sum','WO_QTY':'sum','MIN_AGE':'max','OH_QTY':'sum'}    
    col=['Part_Group','BINCODE','SHIP','INPUT','WO_QTY','MIN_AGE','OH_QTY']
    dfGroup=dtParts.loc[(dtParts['Part_Group']==partNo),
                        col].groupby(['Part_Group','BINCODE']).agg(parms).copy()
                                                 
    
    # 出貨 BIN 數 
    rowShip=dfGroup.loc[((dfGroup['SHIP'] > 0) | (dfGroup['WO_QTY'] > 0)) ].shape    
    # 入庫 BIN 數
    rowInput=dfGroup.loc[(dfGroup['INPUT'] > 0)].shape     
    # 總 BIN 數
    rowCount=dfGroup.shape
    # 目前庫存 BIN 數
    rowNOH=dfGroup.loc[(dfGroup['OH_QTY'] > 0)].shape
    
    # 目前庫存未出貨 BIN 數
    condit=(dfGroup['OH_QTY'] > 0) & (dfGroup['SHIP'] == 0) &  (dfGroup['WO_QTY'] == 0) & (dfGroup['MIN_AGE'] > 15)
    rowRisk_Bin=dfGroup.loc[condit].reset_index()['BINCODE'].tolist()
    rowRisk=dtBin[(dtBin['Part_Group']==partNo) & (dtBin['BINCODE'].isin(rowRisk_Bin))]
    # 目前庫存未出貨數量
    riskQty=rowRisk['OH_QTY'].sum()
        
    del condit
     # 入庫 > 0 出貨 & 工單領用=0 , 入庫 > 15 天 
    condit=(dfGroup['INPUT'] > 0) & (dfGroup['SHIP'] == 0) &  (dfGroup['WO_QTY'] == 0) & (dfGroup['MIN_AGE'] > 15)
    dfGroup=dfGroup.loc[condit]
    dfGroup=dfGroup.reset_index()
        
    # 有入庫未出貨數量
    risk1Qty=dfGroup['INPUT'].sum()
    risk1_Bin=dfGroup['BINCODE'].tolist()

    # 持續入庫 未出貨數量
    rowRisk1=dtBin[(dtBin['Part_Group']==partNo) & (dtBin['BINCODE'].isin(risk1_Bin))]
        
    #Return
    dic={'INPUT_BIN':rowInput[0],'OutPut_BIN':rowShip[0],'BIN_COUNT':rowCount[0],'NOH_BIN':rowNOH[0],
         'RISK_BIN':rowRisk.shape[0],'RISK1_BIN':rowRisk1.shape[0],'riskQty':riskQty,'risk1Qty':risk1Qty}  
    return dic, rowRisk1[['Part_Group','PRODUCTNO','BINCODE','OH_QTY','INPUT']],rowRisk
   
    

#主程式測試 - 原始資料
dtMain.loc[dtMain['Part_Group']=='95.T4014.W0B0B7N']
dfRiskBinS=pd.DataFrame(columns=['PRODUCTNO','BINCODE'])
PART_NO='95.T4014.W0B0B7N'
binc,binList,rowRisk=GetRawCountbyGroup(dtBin,PART_NO)  
binc
binList
dtMain.loc[dtMain['Part_Group']=='95.T4014.W0B0B7N']

In [31]:
import time
tStart = time.time()
dfRiskBinS=pd.DataFrame(columns=['PRODUCTNO','BINCODE'])

GroupParts=dtMain['Part_Group'].unique().tolist()



for PART_NO in GroupParts:
    binc,binList,rowRisk=GetRawCountbyGroup(dtBin,PART_NO)    
    
    #Risk Bin 
    dfRiskBinS=dfRiskBinS.append(binList,sort=False)        
    dtMain.loc[dtMain['Part_Group']==PART_NO,'INPUT_BIN']=binc['INPUT_BIN']
    dtMain.loc[dtMain['Part_Group']==PART_NO,'SHIP_BIN']=binc['OutPut_BIN']
    dtMain.loc[dtMain['Part_Group']==PART_NO,'NOH_BIN']=binc['NOH_BIN']
    dtMain.loc[dtMain['Part_Group']==PART_NO,'RISK_BIN']=binc['RISK_BIN']
    dtMain.loc[dtMain['Part_Group']==PART_NO,'RISK1_BIN']=binc['RISK1_BIN']   
    
    #by 料號計算 Summary
    subParts=dtMain.loc[dtMain['Part_Group']==PART_NO,'PRODUCTNO'].unique().tolist()
    
    for p in subParts:
        riskH=0
        riskSlow=0
        
        riskH=dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'INPUT'].sum()       
        riskSlow=rowRisk.loc[rowRisk['PRODUCTNO']==p,'OH_QTY'].sum()
        
        dtMain.loc[dtMain['PRODUCTNO']==p,'RISK1_QTY']=riskH
        dtMain.loc[dtMain['PRODUCTNO']==p,'RISK_QTY']=riskSlow  
        
        if (riskH > riskSlow): print(p)
    
tEnd = time.time()


dtMain['UNShip_Ratio']=dtMain['RISK1_QTY']/dtMain['OH_QTY']
dtMain['InvFree_Ratio']=dtMain['RISK_QTY']/dtMain['OH_QTY']
dtMain['Total']=dtMain['SHIP']+dtMain['WO_QTY']
print("It cost {:.2} sec".format(tEnd - tStart))            

It cost 7.2e+01 sec


#主程式測試 - 檢驗
dtMain.loc[dtMain['Part_Group']=='95.T4014.W0B0B7N']
dfRiskBinS[dfRiskBinS['Part_Group']=='95.T4014.W0B0B7N']

In [32]:
# 增加拆分 CIE / IV /VF , 新增欄位
dfRiskBinS['PRODUCTNO'].unique()
sTypes=['WL','CIE','IV','VF']
for s in sTypes:
    dfRiskBinS[s]='0'

In [33]:
# 拆碼
pl=ParNameRule['PRODUCTNO'].unique()

for p in pl:
    WLS=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'WL_S'][0])
    WL_E=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'WL_E'][0])

    CIE_S=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'CIE_S'][0])
    CIE_E=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'CIE_E'][0])

    IV_S=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'IV_S'][0])
    IV_E=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'IV_E'][0])

    VF_S=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'VF_S'][0])
    VF_E=int(ParNameRule.loc[ParNameRule['PRODUCTNO']==p,'VF_E'][0])

#print('WL',WLS,WL_E,'CIE',CIE_S,CIE_E,'IV',IV_S,IV_E,'VF',VF_S,VF_E)
    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'CIE']\
    =dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'BINCODE'].str[CIE_S:CIE_E+CIE_S]

    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'WL']=\
    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'BINCODE'].str[WLS:WLS+WL_E]

    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'IV']=\
    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'BINCODE'].str[IV_S:IV_S+IV_E]

    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'VF']=\
    dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==p,'BINCODE'].str[VF_S:VF_S+VF_E]


In [34]:
#串 BU Team

dtBU=(dtBU.assign(rn=dtBU.sort_values(['MODEL'], ascending=False)
                  .groupby(['MODEL']).cumcount() + 1).query('rn <= 1')
      .sort_values(['BG', 'rn']))

dtMain=pd.merge(dtMain,dtBU,how='left', left_on=['MODEL_NAME'] , right_on=['MODEL'])
dtMain=dtMain.drop(columns=['MODEL'],axis=1)
dtMain=dtMain.drop(['rn'],axis=1)
dtMain.BG=dtMain.BG.fillna('OTHERS')

In [35]:
# Merage 導致錯誤
#dfRiskBinS.columns
#dfRiskBinS.head(2)
#dfRiskBinS.loc[dfRiskBinS["BINCODE"]=='F3XX1HQ3']

In [36]:
#dtBin.loc[dtBin["BINCODE"]=='F3XX1HQ3']

In [37]:
# Risk bin merge Model
dfRiskBinS=pd.merge(dfRiskBinS,dtModelPro,how='left', left_on=['PRODUCTNO'] , right_on=['PART_NO'])
dfRiskBinS=dfRiskBinS.drop(['PART_NO'],axis=1)

# dfRiskBinS 增加入庫數量以有增加 20190708
#dfRiskBinS=pd.merge(dfRiskBinS,dtBin[['PRODUCTNO','BINCODE','INPUT']],how='left',left_on=['PRODUCTNO','BINCODE'],right_on=['PRODUCTNO','BINCODE'])

dfRiskBinS=pd.merge(dfRiskBinS,dtBU,how='left', left_on=['MODEL_NAME'] , right_on=['MODEL'])
cols=['BG','MODEL_GROUP','MODEL_NAME','Part_Group','PRODUCTNO','BINCODE','INPUT','OH_QTY','WL', 'CIE', 'IV', 'VF']
dfRiskBinS=dfRiskBinS.reindex(columns=cols)

In [38]:
# Risk Bin 判斷規格是否關閉
dfRiskPart=dfRiskBinS.groupby(['MODEL_NAME','PRODUCTNO'])['INPUT'].min()
# dfRiskPart 料號清單
dfRiskPart=dfRiskPart.reset_index()

In [39]:
# MAIN_BIN 先預設 subMain Bin
dfRiskBinS['MAIN_BIN']='S'
for i in range(0,dfRiskPart.shape[0]):
    # 取得料號 MainBin subMain Bin
    part=dfRiskPart.loc[i,'PRODUCTNO']
    model=dfRiskPart.loc[i,'MODEL_NAME']
    strsql="exec dbo.sp_Get_PM_SPEC_ALL '{0}','{1}'".format(model,part)    
    dfPMSPEC=dbCon.ExecMSSql_Commend(dbCon.lexmsdb2,'Planning',strsql)
    #取 Risk Bin 每個 Bin 逐一檢查
    subRisk=dfRiskBinS.loc[dfRiskBinS['PRODUCTNO']==part]
    for ix in subRisk.index:
        rbin=subRisk.loc[ix,'BINCODE']  
        #print(part,rbin,MinSub)
        MinSub=dfPMSPEC.loc[dfPMSPEC['eng_spec']==rbin,'bin_grade'].tolist()
        if len(MinSub) > 0:
            dfRiskBinS.loc[ix,'MAIN_BIN']=MinSub[0]
        else :
            dfRiskBinS.loc[ix,'MAIN_BIN']='DG'

In [40]:
# Planning 系統關閉的
strSql=sSql.GetTurnOffBin()
dfSMBin=dbCon.ExecMSSql_Commend(dbCon.lexmsdb2,'Planning',strSql)
for i in dfSMBin.index:
    partno=dfSMBin.iloc[i,0]
    bincode=dfSMBin.iloc[i,1]
    smbin=dfRiskBinS.loc[(dfRiskBinS['PRODUCTNO']==partno) & (dfRiskBinS['BINCODE']==bincode),'MAIN_BIN']
    if len(smbin) > 0 :
        dfRiskBinS.loc[(dfRiskBinS['PRODUCTNO']==partno) & (dfRiskBinS['BINCODE']==bincode),'MAIN_BIN']='SYS'
    

Insert DB 另外寫
整理 RPT<BR>
Rename & sorting Columns<BR>
dtMain<BR>
dtBin<BR>
dfRiskBinS<BR>

In [41]:
# dtMain Columns ReName & Sorting
cols={"SHIP":'出貨','WO_QTY':'工單領用','Total':'出貨總量','SHIP_BIN':'下線BIN數','OH_QTY':'FG_庫存','NOH_BIN':'庫存BIN數',
      'INPUT':'FG入庫','INPUT_BIN':'入庫BIN數',
      'RISK1_BIN':'持續入庫\n 未出貨BIN數','RISK1_QTY':'持續入庫\n 未出貨數量','UNShip_Ratio':'未出貨\n 庫存比例',
      'RISK_QTY':'有庫存\n 不合規數量','RISK_BIN':'有庫存\n 不合規BIN數','InvFree_Ratio':'庫存\n 呆滯比例'}

dtMain=dtMain.rename(columns=cols)

# Columns Sorting
cols=['BG','MODEL_GROUP','MODEL_NAME','Part_Group','PRODUCTNO','出貨','工單領用','出貨總量','下線BIN數','FG_庫存','庫存BIN數',
      'FG入庫','入庫BIN數','持續入庫\n 未出貨BIN數','持續入庫\n 未出貨數量','未出貨\n 庫存比例',
      '有庫存\n 不合規數量','有庫存\n 不合規BIN數','庫存\n 呆滯比例']
dtMain=dtMain.reindex(columns=cols)
dtMain=dtMain.fillna(0)

In [42]:
# dtBin Columns ReName & Sortint 
cols=['MODEL_GROUP', 'MODEL_NAME','PRODUCTNO','Part_Group','BINCODE','MIN_AGE'
      ,'SHIP', 'INPUT', 'WO_QTY', 'OH_QTY']
dtBin=dtBin.reindex(columns=cols)

cols={"SHIP":'出貨','WO_QTY':'工單領用','INPUT':'FG入庫','OH_QTY':'FG_庫存'}
dtBin=dtBin.rename(columns=cols)
dtBin=dtBin.fillna(0)

In [43]:
# dtBin Columns ReName & Sortint 
dfRiskBinS=dfRiskBinS.rename(columns={'BINCODE':'持續入庫\n 未出貨BIN',
                                      'OH_QTY':'FG_庫存','INPUT':'FG入庫'})
dfRiskBinS=dfRiskBinS.fillna('Others')

In [44]:
# 排序
dtMain=dtMain.sort_values(by=['持續入庫\n 未出貨數量','FG_庫存'],ascending=False)

In [45]:
#2019/04/16 Add FG入庫 , # Modle Group Summary
dtMGSum=dtMain.groupby(['BG','MODEL_GROUP'],axis=0).agg('sum')
dtMGSum=dtMGSum.sort_values(by=['持續入庫\n 未出貨數量'],ascending=False)
dtMGSum['平均入庫\n BIN 數量']=dtMGSum['FG入庫']/dtMGSum['入庫BIN數']
dtParCount=dtMain.loc[dtMain['FG入庫']>0,['MODEL_GROUP','PRODUCTNO']].groupby(['MODEL_GROUP']).agg('count')
dtParCount=dtParCount.reset_index()
dtMGSum=dtMGSum.reset_index()
dtMGSum=pd.merge(dtMGSum,dtParCount,left_on='MODEL_GROUP',right_on='MODEL_GROUP',how='left')
dtMGSum=dtMGSum.rename(columns={'PRODUCTNO':'30天入庫\n 料號數'})
dtMGSum=dtMGSum.fillna(0)
dtMGSum=dtMGSum.reset_index()
cols=['BG','MODEL_GROUP','FG_庫存','30天入庫\n 料號數','有庫存\n 不合規BIN數','有庫存\n 不合規數量','FG入庫','入庫BIN數',
      '持續入庫\n 未出貨BIN數','持續入庫\n 未出貨數量','平均入庫\n BIN 數量']
dtMGSum=dtMGSum.reindex(columns=cols) 

Excel 處理

In [46]:
import WriteExcel as toExcel

In [47]:
FileName='BinCode下線率_Summary'+EndDate.replace('/','')+'.xlsx'
MyExcel=toExcel.WriteExcel(FileName)

In [48]:
#存儲格格式調整
font=dtMGSum.columns[2:len(dtMGSum.columns)].tolist()
Head_chFormat={}

#Sheet1 ModelGroup
for i in font:   
    Head_chFormat[i]=MyExcel.cell_format_Number

MyExcel.AddSheet(dtMGSum,sheetName='ModelGroupSummary',cell_chFormat=Head_chFormat)
del Head_chFormat   

In [49]:
#Sheet2 Model
dtMain=dtMain.replace([np.inf, -np.inf], 0)
dtMain['平均入庫\n BIN 數量']=dtMain['FG入庫']/dtMain['入庫BIN數']
dtMain=dtMain.fillna(0)

font=dtMain.columns[1:len(dtMain.columns)-2].tolist()
Head_chFormat={}

for i in font:   
    Head_chFormat[i]=MyExcel.cell_format_Number

Head_chFormat["庫存\n 呆滯比例"]=MyExcel.cell_format_Pertange
Head_chFormat["平均入庫\n BIN 數量"]=MyExcel.cell_format_Number

MyExcel.AddSheet(dtMain,sheetName='Part Summary',cell_chFormat=Head_chFormat)
del Head_chFormat

In [50]:
#Sheet3 dfRiskBinS
font=dfRiskBinS.columns[5:len(dfRiskBinS.columns)].tolist()
Head_chFormat={}

for i in font:   
    Head_chFormat[i]=MyExcel.cell_format_Number

MyExcel.AddSheet(dfRiskBinS,sheetName='RiskBin',cell_chFormat=Head_chFormat)
del Head_chFormat

In [51]:
#Sheet4 BinRawData
font=dtBin.columns[5:len(dtBin.columns)].tolist()
Head_chFormat={}

for i in font:   
    Head_chFormat[i]=MyExcel.cell_format_Number

MyExcel.AddSheet(dtBin,sheetName='BinRawData',cell_chFormat=Head_chFormat)
del Head_chFormat

In [52]:
MyExcel.Close()

Mail

In [53]:
def magnify():
    return [dict(selector="th",props=[("background-color", "red"),("font-family", "Calibri"),('color','white')])]

In [54]:
# Mail Header Message BL
HtmlBL_MG=dtMGSum[dtMGSum['BG']=='BL'].sort_values(by=['持續入庫\n 未出貨數量'],ascending=False)
HtmlBL_MG=HtmlBL_MG.iloc[0:10,:]

# Mail Header Message
HtmlBL_Part=dtMain[dtMain['BG']=='BL'].sort_values(by=['持續入庫\n 未出貨數量'],ascending=False)
HtmlBL_Part=HtmlBL_Part.iloc[0:10,:]
HtmlBL_Part=HtmlBL_Part.drop(['出貨','工單領用'],axis=1)

In [55]:
# Mail Header Message LT
HtmlLT_MG=dtMGSum[dtMGSum['BG']=='LT'].sort_values(by=['持續入庫\n 未出貨數量'],ascending=False)
HtmlLT_MG=HtmlLT_MG.iloc[0:10,:]

# Mail Header Message
HtmlLT_Part=dtMain[dtMain['BG']=='LT'].sort_values(by=['持續入庫\n 未出貨數量'],ascending=False)
HtmlLT_Part=HtmlLT_Part.iloc[0:10,:]
HtmlLT_Part=HtmlLT_Part.drop(['出貨','工單領用'],axis=1)

In [56]:
HtmlBL_MG.columns

Index(['BG', 'MODEL_GROUP', 'FG_庫存', '30天入庫\n 料號數', '有庫存\n 不合規BIN數',
       '有庫存\n 不合規數量', 'FG入庫', '入庫BIN數', '持續入庫\n 未出貨BIN數', '持續入庫\n 未出貨數量',
       '平均入庫\n BIN 數量'],
      dtype='object')

In [57]:
HtmlBL_MG=HtmlBL_MG.rename(columns={'MODEL_GROUP':'MODEL<BR>GROUP','30天入庫\n 料號數':'30天入庫<BR>料號數','FG_庫存':'FG<BR>庫存','有庫存\n 不合規數量':'有庫存<BR>不合規數量',
                              '持續入庫\n 未出貨BIN數':'持續入庫<BR>未出貨BIN數','持續入庫\n 未出貨數量':'持續入庫<BR>未出貨數量',
                              '有庫存\n 不合規BIN數':'有庫存<BR>不合規BIN數'
                             })

HtmlLT_MG=HtmlLT_MG.rename(columns={'MODEL_GROUP':'MODEL<BR>GROUP','30天入庫\n 料號數':'30天入庫<BR>料號數','FG_庫存':'FG<BR>庫存','有庫存\n 不合規數量':'有庫存<BR>不合規數量',
                              '持續入庫\n 未出貨BIN數':'持續入庫<BR>未出貨BIN數','FG入庫':'FG入庫','持續入庫\n 未出貨數量':'持續入庫<BR>未出貨數量',
                              '有庫存\n 不合規BIN數':'有庫存<BR>不合規BIN數'
                             })

In [58]:
HtmlBL_Part=HtmlBL_Part.rename(columns={'MODEL_GROUP':'MODEL<BR>GROUP','MODEL_NAME':'MODEL<BR>NAME',
                                  'FG_庫存':'FG<BR>庫存','有庫存\n 不合規數量':'有庫存<BR>不合規數量',
                              '持續入庫\n 未出貨BIN數':'持續入庫<BR>未出貨BIN數','持續入庫\n 未出貨數量':'持續入庫<BR>未出貨數量',
                              '有庫存\n 不合規BIN數':'有庫存<BR>不合規BIN數','未出貨\n 庫存比例':'未出貨<BR>庫存比例',
                               '庫存\n 呆滯比例':'庫存<BR>呆滯比例','下線BIN數':'下線<BR>BIN數','入庫BIN數':'入庫<BR>BIN數' 
                             })
HtmlLT_Part=HtmlLT_Part.rename(columns={'MODEL_GROUP':'MODEL<BR>GROUP','MODEL_NAME':'MODEL<BR>NAME',
                                  'FG_庫存':'FG<BR>庫存','有庫存\n 不合規數量':'有庫存<BR>不合規數量',
                              '持續入庫\n 未出貨BIN數':'持續入庫<BR>未出貨BIN數','持續入庫\n 未出貨數量':'持續入庫<BR>未出貨數量',
                              '有庫存\n 不合規BIN數':'有庫存<BR>不合規BIN數','未出貨\n 庫存比例':'未出貨<BR>庫存比例',
                               '庫存\n 呆滯比例':'庫存<BR>呆滯比例','下線BIN數':'下線<BR>BIN數','入庫BIN數':'入庫<BR>BIN數' 
                             })

In [59]:
# Columns Format Change
cell_chFormat={'FG<BR>庫存':'{:,.0f}','30天入庫<BR>料號數':'{:,.0f}','有庫存<BR>不合規BIN數':'{:,.0f}','有庫存<BR>不合規數量':'{:,.0f}','入庫BIN數':'{:,.0f}',
               '持續入庫<BR>未出貨BIN數':'{:,.0f}','FG入庫':'{:,.0f}', '持續入庫<BR>未出貨數量':'{:,.0f}' ,'平均入庫\n BIN 數量':'{:,.0f}'}
MsgBLMg=HtmlBL_MG.style.format(cell_chFormat).hide_index()
MsgBLMg=MsgBLMg.set_properties(**{'border':'1px solid white','font-family':'Calibri',
                           'text-align': 'right',
                           'background-color': 'rgb(211, 211, 211)'})
MsgBLMg=MsgBLMg.set_table_styles(magnify())

# Columns Format Change
cell_chFormat={'FG<BR>庫存':'{:,.0f}','30天入庫<BR>料號數':'{:,.0f}','有庫存<BR>不合規BIN數':'{:,.0f}','有庫存<BR>不合規數量':'{:,.0f}','入庫BIN數':'{:,.0f}',
               '持續入庫<BR>未出貨BIN數':'{:,.0f}', 'FG入庫':'{:,.0f}','持續入庫<BR>未出貨數量':'{:,.0f}','平均入庫\n BIN 數量':'{:,.0f}' }
MsgLTMg=HtmlLT_MG.style.format(cell_chFormat).hide_index()
MsgLTMg=MsgLTMg.set_properties(**{'border':'1px solid white','font-family':'Calibri',
                           'text-align': 'right',
                           'background-color': 'rgb(211, 211, 211)'})
MsgLTMg=MsgLTMg.set_table_styles(magnify())

In [60]:
# Columns Format Change
cell_chFormat={'出貨總量':'{:,.0f}','下線<BR>BIN數':'{:,.0f}','FG<BR>庫存':'{:,.0f}','庫存<BR>BIN數':'{:,.0f}',
               'FG入庫':'{:,.0f}','入庫<BR>BIN數':'{:,.0f}','持續入庫<BR>未出貨BIN數':'{:,.0f}',
               '持續入庫<BR>未出貨數量':'{:,.0f}','未出貨<BR>庫存比例':'{:.2%}',
               '有庫存<BR>不合規數量':'{:,.0f}','有庫存<BR>不合規BIN數':'{:,.0f}',
               '庫存<BR>呆滯比例':'{:.2%}','平均入庫\n BIN 數量':'{:,.0f}' }
MsgLTPart=HtmlLT_Part.style.hide_index()
MsgLTPart=MsgLTPart.set_properties(**{'border':'1px solid white','font-family':'Calibri',
                           'text-align': 'right',
                           'background-color': 'rgb(211, 211, 211)'})
MsgLTPart=MsgLTPart.format(cell_chFormat).set_table_styles(magnify())

# Columns Format Change
cell_chFormat={'出貨總量':'{:,.0f}','下線<BR>BIN數':'{:,.0f}','FG<BR>庫存':'{:,.0f}','庫存<BR>BIN數':'{:,.0f}',
               'FG入庫':'{:,.0f}','入庫<BR>BIN數':'{:,.0f}','持續入庫<BR>未出貨BIN數':'{:,.0f}',
               '持續入庫<BR>未出貨數量':'{:,.0f}','未出貨<BR>庫存比例':'{:.2%}',
               '有庫存<BR>不合規數量':'{:,.0f}','有庫存<BR>不合規BIN數':'{:,.0f}',
               '庫存<BR>呆滯比例':'{:.2%}','平均入庫\n BIN 數量':'{:,.0f}' }
MsgBLPart=HtmlBL_Part.style.hide_index()
MsgBLPart=MsgBLPart.set_properties(**{'border':'1px solid white','font-family':'Calibri',
                           'text-align': 'right',
                           'background-color': 'rgb(211, 211, 211)'})
MsgBLPart=MsgBLPart.format(cell_chFormat).set_table_styles(magnify())

調整 HTML

In [61]:
strDesc='<span style="font-family:Calibri;">'
strDesc+='各料號細項請參閱副件,Risk bincode By CIE/IV/VF/WD <BR>'
strDesc+='出貨:90D,工單領用:90D,FG入庫:30D <BR>'
strDesc+='下線日期區間:'+StarDate+'-'+EndDate+'<BR>'
strDesc+='F/G入庫區間:'+StarDateM+'-'+EndDate+'<BR>'
strDesc+='<BR>Model Group Summary<BR></span>' 

In [62]:
# Mail Address
strSql=sSql.GetMailList()
dtMailAddr=dbCon.Exec_OracleCommend(dbCon.T01HIS1,strSql)
toAddr=dtMailAddr['MAIL_LIST'].map(lambda x: '{}@lextar.com'.format(x)).tolist()
ccAddr=dtMailAddr.loc[dtMailAddr['MAIL_LIST_CC'].notna(),'MAIL_LIST_CC'].map(lambda x: '{}@lextar.com'.format(x)).tolist()
str=','
toAddr=str.join(toAddr)
ccAddr=str.join(ccAddr)

# Mail 測試
toAddr='Chin.Yang@lextar.com'
ccAddr='Chin.Yang@lextar.com'

In [63]:
sMail=MyMail.MailTools()
Subject='PKG FG 持續入庫未出貨規格'
Msg=strDesc+MsgBLMg.render()+'<BR>'+MsgLTMg.render()+'<BR>Top 10 BL Part'+'<BR>'+MsgBLPart.render()+'<BR>Top 10 LT Part'+'<BR>'+MsgLTPart.render()
sMail.SendAttachFile(Subject,Msg,FileName,toAddr,ccAddr,True)

cc Email sent!


In [64]:
# delete OldFile
DelDate=datetime.now()+relativedelta(days=-7)
DelDate=DelDate.strftime('%Y/%m/%d')
myfile='BinCode下線率_Summary'+DelDate.replace('/','')+'.xlsx'
if os.path.isfile(myfile): os.remove(myfile)    